In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import pathlib 
df = pd.read_csv("../data/raw/student.csv")  # path 
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

num_cols = ["math_score","reading_score","writing_score","hours_studied","attendance_pct"]
cat_cols = ["student_id","gender","class","parent_education","test_preparation"]

for c in num_cols: df[c] = pd.to_numeric(df[c], errors="coerce")
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df[cat_cols] = df[cat_cols].fillna("Unknown")

df["math_score"]     = df["math_score"].clip(0, 100)
df["reading_score"]  = df["reading_score"].clip(0, 100)
df["writing_score"]  = df["writing_score"].clip(0, 100)
df["attendance_pct"] = df["attendance_pct"].clip(0, 100)
df["hours_studied"]  = df["hours_studied"].clip(lower=0)

df["composite"] = df[["math_score","reading_score","writing_score"]].mean(axis=1)

df.to_csv("../data/processed/students_clean.csv", index=False)
# Optional buckets for analysis
df["study_bucket"] = pd.cut(
    df["hours_studied"],
    bins=[-1,1,3,6,100],
    labels=["very_low","low","medium","high"]
)
df["attendance_bucket"] = pd.cut(
    df["attendance_pct"],
    bins=[-1,80,90,100],
    labels=["at_risk","ok","good"]
)

# Risk flag
df["risk_flag"] = np.where(
    (df["attendance_pct"] < 80) | (df["composite"] < 50),
    1, 0
)

# ============== 9) SAVE CLEAN DATA ==============
CLEAN_CSV = "../data/processed/students_clean.csv"
df.to_csv(CLEAN_CSV, index=False)
print(f"[INFO] Cleaned data saved to {CLEAN_CSV}")

# ============== 10) BASIC EDA PRINTS ==============
print("\n===== SHAPE =====")
print(df.shape)

print("\n===== DTYPE INFO =====")
print(df.dtypes)

print("\n===== NUMERIC SUMMARY =====")
print(df[["math_score","reading_score","writing_score","hours_studied","attendance_pct","composite"]].describe().round(2))

print("\n===== MEAN COMPOSITE BY CLASS =====")
by_class = df.groupby("class", dropna=False)["composite"].mean().round(2)
print(by_class)

print("\n===== MEAN COMPOSITE BY TEST PREPARATION =====")
by_prep = df.groupby("test_preparation", dropna=False)["composite"].mean().round(2)
print(by_prep)

print("\n===== RISK % BY CLASS =====")
risk_pct = df.groupby("class", dropna=False)["risk_flag"].mean().mul(100).round(1)
print(risk_pct)

# ============== 11) SAVE TABLES ==============
REPORTS = "../reports"

by_class.to_csv(REPORTS + "/by_class_mean_composite.csv", index=False)
by_prep.to_csv(REPORTS + "/by_testprep_mean_composite.csv", index=False)
risk_pct.to_csv(REPORTS + "/risk_percent_by_class.csv", index=False)


# ============== 12) CHARTS (Matplotlib only) ==============
# Composite distribution
FIGURES = "../figures"
plt.figure()
plt.hist(df["composite"].dropna(), bins=10)
plt.title("Composite Score Distribution")
plt.xlabel("Composite")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(FIGURES+ "/composite_distribution.png", dpi=150)
plt.close()

# Average composite by class (bar chart)
plt.figure()
by_class.plot(kind="bar")
plt.title("Average Composite by Class")
plt.xlabel("Class")
plt.ylabel("Average Composite")
plt.tight_layout()
plt.savefig(FIGURES + "/avg_composite_by_class.png", dpi=150)
plt.close()

# Hours studied vs composite (scatter)
plt.figure()
plt.scatter(df["hours_studied"], df["composite"])
plt.title("Hours Studied vs Composite")
plt.xlabel("Hours Studied")
plt.ylabel("Composite")
plt.tight_layout()
plt.savefig(FIGURES + "/hours_vs_composite.png", dpi=150)
plt.close()

# Attendance vs composite (scatter)
plt.figure()
plt.scatter(df["attendance_pct"], df["composite"])
plt.title("Attendance % vs Composite")
plt.xlabel("Attendance %")
plt.ylabel("Composite")
plt.tight_layout()
plt.savefig(FIGURES + "/attendance_vs_composite.png", dpi=150)
plt.close()

print(f"\n[INFO] Charts saved to {FIGURES}")
print("[DONE] Pipeline finished successfully.")


[INFO] Cleaned data saved to ../data/processed/students_clean.csv

===== SHAPE =====
(6, 14)

===== DTYPE INFO =====
student_id             object
gender                 object
class                  object
math_score              int64
reading_score           int64
writing_score           int64
hours_studied         float64
attendance_pct          int64
parent_education       object
test_preparation       object
composite             float64
study_bucket         category
attendance_bucket    category
risk_flag               int64
dtype: object

===== NUMERIC SUMMARY =====
       math_score  reading_score  writing_score  hours_studied  \
count        6.00           6.00           6.00           6.00   
mean        68.67          73.00          70.83           3.33   
std         19.49          15.74          18.16           2.68   
min         40.00          50.00          45.00           0.50   
25%         57.75          63.00          61.00           1.25   
50%         69.00       